# DictFS

Turn a dictionary into a file system.

In [84]:
from fs.base import FS
from fs.subfs import SubFS
from fs.info import Info
from os.path import basename, dirname
from fs.errors import ResourceNotFound, ResourceReadOnly
from fs.errors import RemoveRootError
from fs.errors import DirectoryExists, DirectoryNotEmpty
from fs.errors import FileExpected, DirectoryExpected
from io import BytesIO
from Dcel import Dcel

DirectoryTypes = (dict,list,Dcel)
ByteableTypes = (bytes,str)
FlexibleTypes = (Dcel,)
WriteModes = ('w','a')

class DictFS(FS):
    def __init__(self,fsdict,mode='a'):
        if(type(fsdict) is str):
            fsdict = { fsdict: fsdict }
        self.fsdict = fsdict
        self._mode = mode
        super().__init__()
    
    @property
    def _addr(self):
        return self.fsdict
    
    def _pathwalk(self,
                  path,
                  target=None
                 ):
        if target is None:
            target = self.fsdict
            
        if (not type(target) is dict):
            # UPDATE 8/7/2022 - raygan - Return fsdict without lookup.
            # This enables Fudge glob where fu/'*' needs to return a list.
            if (type(target) is Dcel
                or str(type(target)) == "<class '__main__.Dcel'>"):
                return target._pathwalk(path)
            else:
                raise TypeError(f"DictFS internal 'fsdict' type must 'dict' or 'Dcel' not {type(target)}.")
        
        if path in (None,"",".","/"):
            return target
        
        path = path.strip("/")

        split_result = path.split('/',1)
        if len(split_result) == 2:
            seg,nextpath = split_result
        else:
            seg = split_result[0]
            nextpath = None

        # lookup seg within target.
        # target may be a Dict or Dcel
        if not seg in target:
            raise ResourceNotFound(path)

        nexttarget = target[seg]

        if ((type(nexttarget) is dict
            or type(nexttarget) is Dcel)
            and not nextpath is None):
            return self._pathwalk(nextpath,
                                  nexttarget)
        return nexttarget
    
    
    ####################
    # pyfilesystem Api #
    ####################
    
    def getinfo(self, path, namespaces=None):
        is_dir = self.isdir(path)
            
        i = Info({"basic":{
            "name": basename(path.strip('/')),
            "is_dir": is_dir
        }})
        
        if namespaces != None \
        and "dcel" in namespaces:
            target = self._pathwalk(path)
            i.raw["dcel"] = {"value":
                             target
                            }
        return i
    
    def listdir(self, path):
        if self.isdir(path):
            target = self._pathwalk(path)
            if type(target) != type(None):
                try:
                    # This might fail if Dcel::iter() doesn't work.
                    res = list(target)
                except:
                    res = None
                if type(res) == type(None):
                    return []
                return list(target)
            else:
                return []
        raise DirectoryExpected(path)
    
    def isdir(self, path='/'):
        target = self._pathwalk(path)
        _type = type(target)
        
        if _type in DirectoryTypes:
            return True
        if _type in FlexibleTypes:
            return target.isdir(path)
            # short circuited
            _value = target.value
            if (hasattr(_value,'__getitem__')
               and not issubclass(_type,str)):
                return True
        else:
            return False
    
    def makedir(self, path, permissions=None, recreate=False):
        
        if not self._mode in WriteModes:
            # raise ReadOnlyFilesystem
            return
        
        parpath = dirname(path)
        entryname = basename(path)
        target = self._pathwalk(parpath)
        if target is None:
            raise ResourceNotFound
            
        if entryname in target\
        and recreate is False:
            raise DirectoryExists(path)
        
        target[entryname] = dict()
        
        return SubFS(self, path)
    
    def openbin(self, path, mode='r', buffering=-1, **options):
        
        if mode in WriteModes\
        and not self._mode in WriteModes:
            raise ResourceReadOnly(f"DictFS: {path}")
        parpath = dirname(path)
        entryname = basename(path)
        parent = self._pathwalk(parpath)
        target = parent[entryname]
        targettype = type(target)
        
        if targettype in DirectoryTypes:
            raise FileExpected(path)
        
        if targettype is str:
            buf = target.encode(encoding='utf-8')
            # kludge to get writebytes to work
            # but it messes up getting strings from DictFS
            # parent[entryname] = buf
        elif targettype is bytes:
            buf = target
            
        else:
            raise FileExpected(path)
        
        return BytesIO(buf)
    
    def remove(self, path):
        if not self._mode in WriteModes:
            # raise ReadOnlyFilesystem
            return
        
        parpath = dirname(path)
        entryname = basename(path)
        pardir = self._pathwalk(parpath)
        if entryname in pardir:
            if not type(pardir[entryname]) in DirectoryTypes:
                pardir.pop(entryname)
            else:
                raise FileExpected(path)
        else:
            raise ResourceNotFound(path)
        
    def removedir(self, path):
        if not self._mode in WriteModes:
            # raise ReadOnlyFilesystem
            return
        
        if path == "/":
            raise RemoveRootError
            
        parpath = dirname(path)
        entryname = basename(path)
        pardir = self._pathwalk(parpath)
        if entryname in pardir:
            if type(pardir[entryname]) in DirectoryTypes:
                if pardir[entryname]:
                    raise DirectoryNotEmpty(path)
                else:
                    pardir.pop(entryname)
            else:
                raise DirectoryExpected(path)
        else:
            raise ResourceNotFound(path)
        
        
    def setinfo(self, path, info):
        if not self._mode in WriteModes:
            # raise ReadOnlyFilesystem
            return
        try:
            parpath = dirname(path)
            entryname = basename(path)
            target = self._pathwalk(parpath)
        except:
            return
        try:
            value = info['dcel']['value']
            target[entryname] = value
        except:
            return
        
    ### pyfilesystem:
    #   override methods
    
    def writetext(self,
                  path,
                  contents,
                  encoding='utf-8',
                  errors=None,
                  newline=''):
        if not self._mode in WriteModes:
            raise ResourceReadOnly
        try:
            # walk to target
            parpath = dirname(path)
            entryname = basename(path)
            target = self._pathwalk(parpath)
        except:
            return
        try:
            # set target
            result = target[entryname]
            if type(result) is Dcel:
                result.value = contents
            else:
                target[entryname] = contents
        except:
            return
    
    ####################
    #  cosmos API      #
    ####################
    
    # Depricate external use of any API other than pyfilesystem.
    # OK to keep if this is internal.

    def lookup(self, path, base=None):
        return self._pathwalk(path, base)
        

In [38]:
strdictfs = DictFS('hello')

In [39]:
print(strdictfs.readtext('/hello'))

hello


In [40]:

dcel = Dcel({'a':'Aye'})
dfs = DictFS(dcel)
dfs_cel = Dcel(address=dcel, service_class=DictFS)


In [41]:
dfs_cel.getinfo('a')

<dir 'a'>

In [42]:
dfs_cel.isdir('a')

True

In [43]:
dfs_cel.listdir('/')

DictFS::listdir(): path = /


['a']

In [44]:
res = "a/b".split('/',1)
if len(res) == 2:
    a,b = res
else:
    a = res[0]
    b = None
    
print(f"{a} {b}")

a b


In [45]:
temp_path = '/A//'
temp_path.strip('/').split('/',1)

['A']

### Test: Use Dcel as dict input for DictFS

In [46]:
from Dcel import Dcel

In [47]:
dict_dcel = Dcel({"a": {"Aye": "A"}})
dictfs_dcel = Dcel(address=dict_dcel, service_class=DictFS)

In [48]:
print(dictfs_dcel.inspect())
print(dictfs_dcel.service.exists('/a/Aye/A/B'))
print(dictfs_dcel.service.getinfo('a/Aye'))
print(dictfs_dcel.service._pathwalk('a/Aye'))
print('a' in dictfs_dcel)
print(dictfs_dcel.service.fsdict['a']['Aye'])

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class '__main__.DictFS'>:<__main__.DictFS object at 0x7f2cf73a29b0>
        value: <class 'Dcel.Dcel'>:{'a': {'Aye': 'A'}}
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        
False
<dir 'Aye'>
A
True
A


In [49]:
print(dict_dcel.inspect())

address: Address attribute not present.:
        abspath: Abspath calulated from `None`.:
        service: Service attribute not present.:
        value: <class 'dict'>:{'a': {'Aye': 'A'}}
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [50]:
print(dictfs_dcel.inspect())

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class '__main__.DictFS'>:<__main__.DictFS object at 0x7f2cf73a29b0>
        value: <class 'Dcel.Dcel'>:{'a': {'Aye': 'A'}}
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [51]:
### FIXME: DictFS should return True

print(dict_dcel.isdir())
print(dictfs_dcel.isdir())
assert(dict_dcel.isdir() == dictfs_dcel.isdir())

True
True


In [52]:
dictfs_dcel.service.fsdict

In [53]:
dictfs_dcel.service._pathwalk('/a/Aye')

In [54]:
print(dictfs_dcel['a'])

{'Aye': 'A'}


In [55]:
print(dictfs_dcel.inspect())
print(dictfs_dcel.address)
print(dictfs_dcel.isdir('/'))
print(dictfs_dcel._dir)

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class '__main__.DictFS'>:<__main__.DictFS object at 0x7f2cf73a29b0>
        value: <class 'Dcel.Dcel'>:{'a': {'Aye': 'A'}}
        _map: <class 'NoneType'>:None
        _dir: <class 'dict'>:{'a': <Dcel.Dcel object at 0x7f2cf73a3370>}
        
/
True
{'a': <Dcel.Dcel object at 0x7f2cf73a3370>}


In [56]:
print(dictfs_dcel.service.exists('blue'))

KeyError: 'blue'

### Test geturl()

In [57]:
# temp:
print(type(0))
d = {"a":{"Aye":"A"}}
l = list(d.keys())
print(l)

print(dict_dcel.keys())
#l2 = list(dict_dcel)
for each in d:
    print(each)

<class 'int'>
['a']
dict_keys(['a'])
a


In [58]:
s = DictFS({"a":{"apple":"True","apricot":"True","anchovy":"True"}})
print(s.geturl('/a/apple'))
print(s.getinfo('/a/apple').raw)
print(s.readtext('/a/apple'))

NoURL: path '/a/apple' has no 'download' URL

In [59]:
# Test writethrough with recursive DictFS
from Dcel import Dcel
baselayer = Dcel("apple a b c carrots")
apple = baselayer[0:5]
abc = baselayer[6:11]
carrots = baselayer[12:]
print(f"{apple} {abc} {carrots}")

based = Dcel({"apple": apple,
              "abc": abc,
              "carrots": carrots
             },
             service_class=DictFS
            )
print(based.service.fsdict)

a = abc[0:1]
b = abc[2:3]
c = abc[4:5]
print(f"{a} {b} {c}")

abcd = Dcel({"a":a, "b":b, "c":c},
         service_class=DictFS)
print(abcd.service.fsdict)

abcd['b'] = 'balloon'
print(abcd)
print(baselayer)

apple a b c carrots
{'apple': <Dcel.Dcel object at 0x7f2cf73a2e60>, 'abc': <Dcel.Dcel object at 0x7f2cf73a3730>, 'carrots': <Dcel.Dcel object at 0x7f2cf73a3340>}
a b c
{'a': <Dcel.Dcel object at 0x7f2cf41e62f0>, 'b': <Dcel.Dcel object at 0x7f2cf41e5b70>, 'c': <Dcel.Dcel object at 0x7f2cf41e57b0>}
{'a': <Dcel.Dcel object at 0x7f2cf41e62f0>, 'b': <Dcel.Dcel object at 0x7f2cf41e5b70>, 'c': <Dcel.Dcel object at 0x7f2cf41e57b0>}
apple a balloon c carrots


In [60]:
abcd['b'].flush()
print(f"{abcd['b'].address} {abcd['b'].service}")

abcd['b'].value.flush()


/b <__main__.DictFS object at 0x7f2cf41e6cb0>


In [61]:
baselayer.value.flush()

AttributeError: 'str' object has no attribute 'flush'

In [62]:
# Test dict with Dcels in the values.
from Dcel import Dcel

a = Dcel("apple")
b = a[1:3]  # test with Dcel slice
c = Dcel("carrot")

stringbuf = Dcel("apple carrot")
a = stringbuf[0:5]
b = a[1:3]  # test with Dcel slice
c = stringbuf[6:12]
fsdict = { "a": a, "b": b, "c": c }

d = Dcel(fsdict,service_class=DictFS)
print(d["b"].address)
print(type(d["b"]))
d["b"] = "bb"  # this should access the slice

for ea in d.listdir():
   print(f"{ea}: {d[ea]} {type(d[ea])}")

print(stringbuf)


/b
<class 'Dcel.Dcel'>
DictFS::listdir(): path = /
a: abble <class 'Dcel.Dcel'>
b: bb <class 'Dcel.Dcel'>
c: carrot <class 'Dcel.Dcel'>
abble carrot


In [63]:
# test data
from fs.osfs import OSFS
fsdict = { "colors":
          { "red": "Rose",
            "green": "Grapes",
            "blue": "Berries"
           },
           "services": { "file": OSFS }
         }

a = DictFS(fsdict)

In [64]:
# lookup()
a.lookup("colors/blue")

'Berries'

In [65]:
# exists()
print(a.exists("colors/blue"))
print(a.exists('/' + '/'.lstrip('/')))

True
True


In [66]:
# isdir()
paths = ['/','colors','colors/blue']
for each in paths:
    print(a.isdir(each))


True
True
False


In [67]:
# readtext()
res = a.lookup('/colors/blue')
print(res)

Berries


In [68]:
from Dcel import Dcel
d = Dcel(fsdict, service_class=DictFS)

In [69]:
print(d.value['colors'])

{'red': 'Rose', 'green': 'Grapes', 'blue': 'Berries'}


In [70]:
d.getinfo('/').raw
#a.readtext('/')
#a._addr

{'basic': {'name': '', 'is_dir': True}}

In [71]:
res = d["colors/red"]
print(type(res))
print(res)

<class 'Dcel.Dcel'>
Rose


In [72]:
print(a.readtext("colors/red"))

Rose


In [73]:
a.writetext("colors/red","Fire Truck")

In [74]:
a.remove("colors/red")

In [75]:
a.listdir('///colors/')

DictFS::listdir(): path = ///colors/


['green', 'blue']

In [76]:
a.getinfo('/services/file',namespaces=['dcel']).raw

{'basic': {'name': 'file', 'is_dir': False}, 'dcel': {'value': fs.osfs.OSFS}}

In [77]:
a.makedir('/colors/grey')
a.makedir('/colors/brown')

SubFS(<__main__.DictFS object at 0x7f2cf73a2f80>, '/colors/brown')

In [78]:
a.removedir('colors/grey')

In [79]:
"//red//green/blue".strip('/').split('/',1)

['red', '/green/blue']

In [80]:
from os.path import basename, dirname

basename("red/green//")
dirname("/blue/yellow")


'/blue'

In [81]:
d = {1:2,3:4}
d.pop(3)
d

{1: 2}

In [82]:
l = []
if l:
    print(True)
else:
    print(False)

False
